In [1]:
import pandas as pd
import numpy as np
import joblib
import matplotlib.pyplot as plt
from pylab import rcParams

from src.data.loaders import CommodityLoader
from src.utils.split_series import split_series
from src.evaluation.evaluation import MetricEvaluator

In [2]:
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

plt.style.use('fivethirtyeight')
plt.rcParams['lines.linewidth'] = 1.5
dark_style = {
    'figure.facecolor': '#212946',
    'axes.facecolor': '#212946',
    'savefig.facecolor':'#212946',
    'axes.grid': True,
    'axes.grid.which': 'both',
    'axes.spines.left': False,
    'axes.spines.right': False,
    'axes.spines.top': False,
    'axes.spines.bottom': False,
    'grid.color': '#2A3459',
    'grid.linewidth': '1',
    'text.color': '0.9',
    'axes.labelcolor': '0.9',
    'xtick.color': '0.9',
    'ytick.color': '0.9',
    'font.size': 12 
}
plt.rcParams.update(dark_style)

In [ ]:
df = pd.read_parquet("model_predictions/oracle/ARIMA_ML_LSTM_oracle.parquet")
df

In [3]:
df_brl = CommodityLoader.load_all_commodities(currency='BRL', preprocessing=True, monthly_aggregation='mean', limit_date=None)

ദ്ദി・ᴗ・)✧ acucar_santos carregado com sucesso
ദ്ദി・ᴗ・)✧ acucar_sp carregado com sucesso
ദ്ദി・ᴗ・)✧ algodao carregado com sucesso
ദ്ദി・ᴗ・)✧ arroz carregado com sucesso
ദ്ദി・ᴗ・)✧ cafe_arabica carregado com sucesso
ദ്ദി・ᴗ・)✧ cafe_robusta carregado com sucesso
ദ്ദി・ᴗ・)✧ milho carregado com sucesso
ദ്ദി・ᴗ・)✧ soja_parana carregado com sucesso
ദ്ദി・ᴗ・)✧ soja_paranagua carregado com sucesso
ദ്ദി・ᴗ・)✧ trigo_parana carregado com sucesso
ദ്ദി・ᴗ・)✧ trigo_rs carregado com sucesso


In [4]:
df_brl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2879 entries, 0 to 2878
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   ds         2879 non-null   datetime64[ns]
 1   y          2879 non-null   float64       
 2   unique_id  2879 non-null   object        
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 67.6+ KB


In [5]:
saved_models = {
    'ARIMA': {
        'ACUCAR_SANTOS': '../saved/arima_tunned_ACUCAR_SANTOS.pkl',
        'ACUCAR_SP': '../saved/arima_tunned_ACUCAR_SP.pkl',
        'ALGODAO': '../saved/arima_tunned_ALGODAO.pkl',
        'ARROZ': '../saved/arima_tunned_ARROZ.pkl',
        'CAFE_ARABICA': '../saved/arima_tunned_CAFE_ARABICA.pkl',
        'CAFE_ROBUSTA': '../saved/arima_tunned_CAFE_ROBUSTA.pkl',
        'MILHO': '../saved/arima_tunned_MILHO.pkl',
        'SOJA_PARANA': '../saved/arima_tunned_SOJA_PARANA.pkl',
        'SOJA_PARANAGUA': '../saved/arima_tunned_SOJA_PARANAGUA.pkl',
        'TRIGO_PARANA': '../saved/arima_tunned_TRIGO_PARANA.pkl',
        'TRIGO_RS': '../saved/arima_tunned_TRIGO_RS.pkl',
    },
    'MLForecast': {
        'svr': '../saved/svr_tunned.pkl',
        'mlp': '../saved/mlp_tunned.pkl',
        'rf': '../saved/rf_tunned.pkl',
        'xgb': '../saved/xgb_tunned.pkl',
        'lgbm': '../saved/lgbm_tunned.pkl',
    }
}

In [24]:
model = joblib.load('../saved/svr_tunned.pkl')
print(model.models)

{'svr': Pipeline(steps=[('minmaxscaler', MinMaxScaler()),
                ('svr', SVR(C=100, gamma=0.1, kernel='linear'))])}


In [6]:
evaluator = MetricEvaluator()

lags = [1,2,3,4,5,6,7,8,9,10,11,12]
lag_window_size = len(lags)

### MLForecast predictions

In [63]:
keys = saved_models['MLForecast'].keys()

In [64]:
df_oracle_temp = df_brl.copy()

for k in keys:
    df_oracle_temp[k] = np.nan

df_oracle_temp.head()

,ds,y,unique_id,svr,mlp,rf,xgb,lgbm
0,2020-01-01,73.428636,ACUCAR_SANTOS,NaN,NaN,NaN,NaN,NaN
1,2020-02-01,81.636111,ACUCAR_SANTOS,NaN,NaN,NaN,NaN,NaN
2,2020-03-01,79.906364,ACUCAR_SANTOS,NaN,NaN,NaN,NaN,NaN
3,2020-04-01,77.113500,ACUCAR_SANTOS,NaN,NaN,NaN,NaN,NaN
4,2020-05-01,84.638500,ACUCAR_SANTOS,NaN,NaN,NaN,NaN,NaN


In [65]:
train_mlf, val_mlf, test_mlf = split_series(df_brl)
full_train_mlf = pd.concat([train_mlf, val_mlf])

Treino + Val ACUCAR_SANTOS: 45 | Teste ACUCAR_SANTOS: 20
Treino + Val ACUCAR_SP: 185 | Teste ACUCAR_SP: 80
Treino + Val ALGODAO: 243 | Teste ALGODAO: 105
Treino + Val ARROZ: 168 | Teste ARROZ: 72
Treino + Val CAFE_ARABICA: 241 | Teste CAFE_ARABICA: 104
Treino + Val CAFE_ROBUSTA: 201 | Teste CAFE_ROBUSTA: 87
Treino + Val MILHO: 175 | Teste MILHO: 75
Treino + Val SOJA_PARANA: 234 | Teste SOJA_PARANA: 101
Treino + Val SOJA_PARANAGUA: 161 | Teste SOJA_PARANAGUA: 70
Treino + Val TRIGO_PARANA: 179 | Teste TRIGO_PARANA: 77
Treino + Val TRIGO_RS: 179 | Teste TRIGO_RS: 77
Splitted data into TRAIN with size 1605, VALIDATION with size 406 and TEST with size 868


In [66]:
test_mlf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 868 entries, 0 to 867
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   ds         868 non-null    datetime64[ns]
 1   y          868 non-null    float64       
 2   unique_id  868 non-null    object        
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 20.5+ KB


In [67]:
def generate_forecasts_dict(keys, saved_models, test_mlf):
    forecasts_dict = {}
    test_mlf = test_mlf.sort_values(['unique_id', 'ds']).reset_index(drop=True)

    for k in keys:
        saved_path = saved_models['MLForecast'][k]
        model = joblib.load(saved_path)
        print(f"MLForecast: Starting prediction of model {k} | {saved_path}")

        forecasts = []

        for i in range(len(test_mlf)):
            f_next = model.predict(h=1)
            forecasts.append(f_next)
            obs = test_mlf.iloc[[i]][['unique_id','ds','y']]
            model.update(obs)

        forecasts_df = pd.concat(forecasts, ignore_index=True)
        forecasts_dict[k] = forecasts_df

    return forecasts_dict

In [68]:
forecasts_dict = generate_forecasts_dict(keys, saved_models, test_mlf)

MLForecast: Starting prediction of model svr | ../saved/svr_tunned.pkl
MLForecast: Starting prediction of model mlp | ../saved/mlp_tunned.pkl
MLForecast: Starting prediction of model rf | ../saved/rf_tunned.pkl
MLForecast: Starting prediction of model xgb | ../saved/xgb_tunned.pkl
MLForecast: Starting prediction of model lgbm | ../saved/lgbm_tunned.pkl


/home/hiraeth/anaconda3/envs/smps/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/home/hiraeth/anaconda3/envs/smps/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/home/hiraeth/anaconda3/envs/smps/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/home/hiraeth/anaconda3/envs/smps/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/home/hiraeth/anaconda3/envs/smps/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but L

In [69]:
forecasts_dict.keys()

dict_keys(['svr', 'mlp', 'rf', 'xgb', 'lgbm'])

In [70]:
forecasts_dict['svr'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9548 entries, 0 to 9547
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   unique_id  9548 non-null   object        
 1   ds         9548 non-null   datetime64[ns]
 2   svr        9548 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 223.9+ KB


In [71]:
# for model, data in forecasts_dict.items():
#     data.to_parquet(f'{model}_prediction.parquet', index=False)

In [72]:
svr = forecasts_dict['svr']
mlp = forecasts_dict['mlp']
rf = forecasts_dict['rf']
xgb = forecasts_dict['xgb']
lgbm = forecasts_dict['lgbm']

In [73]:
svr  = svr[['unique_id', 'ds', 'svr']]
mlp  = mlp[['mlp']]
rf   = rf[['rf']]
xgb  = xgb[['xgb']]
lgbm = lgbm[['lgbm']]

merged = pd.concat([svr, mlp, rf, xgb, lgbm], axis=1)

In [74]:
merged_drop = merged.drop_duplicates(ignore_index=True)

In [75]:
df_merged = pd.merge(
    df_brl,
    merged_drop,
    on=["unique_id", "ds"],
    how="left"
)

In [76]:
colls_models = ['svr', 'mlp', 'rf', 'xgb', 'lgbm']

def choose_oracle(row):
    valids = row[colls_models].dropna()
    
    if valids.empty:
        return None
    
    errs = (valids - row['y']).abs()
    best_model = errs.idxmin()
    
    return valids[best_model]

df_merged['Oracle'] = df_merged.apply(choose_oracle, axis=1)

In [77]:
# df_merged.to_parquet('ML_oracle.parquet', index=False)

In [78]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2879 entries, 0 to 2878
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   ds         2879 non-null   datetime64[ns]
 1   y          2879 non-null   float64       
 2   unique_id  2879 non-null   object        
 3   svr        868 non-null    float64       
 4   mlp        868 non-null    float64       
 5   rf         868 non-null    float64       
 6   xgb        868 non-null    float32       
 7   lgbm       868 non-null    float64       
 8   Oracle     868 non-null    float64       
dtypes: datetime64[ns](1), float32(1), float64(6), object(1)
memory usage: 191.3+ KB


### ARIMA Predictions

In [79]:
arima_preds = pd.read_parquet("model_predictions/arima_prediction.parquet")
arima_preds.head()

,unique_id,order,seasonal_order,y_true,y_pred,ds
0,ACUCAR_SANTOS,"[0, 1, 0]","[0, 0, 0, 12]",176.677619,176.462642,2023-10-01
1,ACUCAR_SANTOS,"[0, 1, 0]","[0, 0, 0, 12]",172.731500,178.971537,2023-11-01
2,ACUCAR_SANTOS,"[0, 1, 0]","[0, 0, 0, 12]",149.326842,174.889769,2023-12-01
3,ACUCAR_SANTOS,"[0, 1, 0]","[0, 0, 0, 12]",154.029545,150.941207,2024-01-01
4,ACUCAR_SANTOS,"[0, 1, 0]","[0, 0, 0, 12]",157.357368,155.708223,2024-02-01


In [80]:
arima_preds.drop(['order', 'seasonal_order', 'y_true'], axis=1, inplace=True)
arima_preds.rename(columns={'y_pred': 'ARIMA'}, inplace=True)
arima_preds.head()

,unique_id,ARIMA,ds
0,ACUCAR_SANTOS,176.462642,2023-10-01
1,ACUCAR_SANTOS,178.971537,2023-11-01
2,ACUCAR_SANTOS,174.889769,2023-12-01
3,ACUCAR_SANTOS,150.941207,2024-01-01
4,ACUCAR_SANTOS,155.708223,2024-02-01


In [81]:
df_merged = pd.merge(
    df_merged,
    arima_preds,
    on=["unique_id", "ds"],
    how="left"
)

In [82]:
df_merged.head()

,ds,y,unique_id,svr,mlp,rf,xgb,lgbm,Oracle,ARIMA
0,2020-01-01,73.428636,ACUCAR_SANTOS,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-01,81.636111,ACUCAR_SANTOS,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-03-01,79.906364,ACUCAR_SANTOS,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-04-01,77.113500,ACUCAR_SANTOS,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-05-01,84.638500,ACUCAR_SANTOS,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [84]:
colls_models = ['svr', 'mlp', 'rf', 'xgb', 'lgbm', 'ARIMA']

df_merged['Oracle'] = df_merged.apply(choose_oracle, axis=1)

In [89]:
reorder = ['unique_id', 'ds', 'y', 'ARIMA', 'svr', 'mlp', 'rf', 'xgb', 'lgbm', 'Oracle']

df_merged = df_merged[reorder]

In [91]:
df_merged.head()

,unique_id,ds,y,ARIMA,svr,mlp,rf,xgb,lgbm,Oracle
0,ACUCAR_SANTOS,2020-01-01,73.428636,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ACUCAR_SANTOS,2020-02-01,81.636111,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ACUCAR_SANTOS,2020-03-01,79.906364,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ACUCAR_SANTOS,2020-04-01,77.113500,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ACUCAR_SANTOS,2020-05-01,84.638500,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [92]:
df_merged.to_parquet('ARIMA_ML_oracle.parquet', index=False)